# Hospital overcapacity

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get state hospital utilization reports

In [3]:
before = pd.read_csv(
    "../hospitals/output/hospital_utilization.csv", dtype={"fac_no": str}
)

In [4]:
before = before[before["icu_beds"] > 0].sort_values("fac_name", ascending=True)

In [5]:
state_hospital_names = before["fac_name"].to_list()

In [6]:
before.head(10)

,fac_no,fac_name,fac_city,county,license_no,fac_zip,icu_beds,tot_lic_beds
488,106580996,ADVENTIST HEALTH AND RIDEOUT,MARYSVILLE,Yuba,230000126,95901,24,221
77,106150788,ADVENTIST HEALTH BAKERSFIELD,BAKERSFIELD,Kern,120000187,93301,34,254
85,106171049,ADVENTIST HEALTH CLEARLAKE,CLEARLAKE,Lake,110000174,95422,4,32
121,106190323,ADVENTIST HEALTH GLENDALE,GLENDALE,Los Angeles,930000059,91206,42,515
84,106164029,ADVENTIST HEALTH HANFORD,HANFORD,Kings,550001439,93230,22,173
209,106234038,ADVENTIST HEALTH HOWARD MEMORIAL,WILLITS,Mendocino,110000013,95490,4,25
381,106390923,ADVENTIST HEALTH LODI MEMORIAL,LODI,San Joaquin,030000056,95240-5179,10,194
482,106560525,ADVENTIST HEALTH SIMI VALLEY,SIMI VALLEY,Ventura,050000216,93065,24,144
474,106554011,ADVENTIST HEALTH SONORA - GREENLEY,SONORA,Tuolumne,030000094,95370,6,140
221,106281078,ADVENTIST HEALTH ST. HELENA,ST. HELENA,Napa,110000073,94574,12,151


### Get latest hospital capacity report from HHS

In [7]:
current = pd.read_csv("output/current_ca.csv")

In [8]:
current = current[current["total_covid_patients"] > 0].sort_values(
    "hospital", ascending=True
)

In [9]:
current["hospital"] = current["hospital"].str.upper()

In [10]:
hhs_hospital_names = current["hospital"].to_list()

In [11]:
current.head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
198,ADVENTIST HEALTH AND RIDEOUT,Short Term,50133.0,2021-01-01,Yuba,115,28.0,17.0,2.0,0.93,76.0,215.0,0.35,Greater Sacramento
101,ADVENTIST HEALTH BAKERSFIELD,Short Term,50455.0,2021-01-01,Kern,29,29.0,21.0,1.0,0.97,123.0,230.0,0.53,San Joaquin Valley
293,ADVENTIST HEALTH CLEARLAKE,Critical Access Hospitals,51317.0,2021-01-01,Lake,33,4.0,1.0,0.0,1.00,2.0,21.0,0.10,Northern California
38,ADVENTIST HEALTH DELANO,Short Term,50608.0,2021-01-01,Kern,29,11.0,8.0,1.0,0.91,27.0,40.0,0.68,San Joaquin Valley
86,ADVENTIST HEALTH HANFORD,Short Term,50121.0,2021-01-01,Kings,31,24.0,14.0,2.0,0.92,94.0,164.0,0.57,San Joaquin Valley
280,ADVENTIST HEALTH HOWARD MEMORIAL,Critical Access Hospitals,51310.0,2021-01-01,Mendocino,45,4.0,3.0,0.0,1.00,3.0,21.0,0.14,Northern California
136,ADVENTIST HEALTH LODI MEMORIAL,Short Term,50336.0,2021-01-01,San Joaquin,77,20.0,7.0,2.0,0.90,49.0,106.0,0.46,San Joaquin Valley
307,ADVENTIST HEALTH REEDLEY,Short Term,50192.0,2021-01-01,Fresno,19,0.0,0.0,0.0,NaN,1.0,19.0,0.05,San Joaquin Valley
148,ADVENTIST HEALTH SIMI VALLEY,Short Term,50236.0,2021-01-01,Ventura,111,24.0,18.0,3.0,0.88,59.0,133.0,0.44,Southern California
255,ADVENTIST HEALTH SONORA,Short Term,50335.0,2021-01-01,Tuolumne,109,6.0,1.0,1.0,0.83,11.0,49.0,0.22,San Joaquin Valley


---

In [12]:
len(before)

345

In [13]:
len(current)

316

In [14]:
hhs_merge = pd.merge(
    current, before, left_on="hospital", right_on="fac_name", how="left"
)

In [15]:
state_merge = pd.merge(
    current, before, left_on="hospital", right_on="fac_name", how="right"
)

In [16]:
len(state_merge)

345

In [17]:
len(hhs_merge)

317

In [18]:
state_merge[["hospital", "fac_name"]].to_csv("output/state_names.csv", index=False)

In [19]:
hhs_merge[["hospital", "fac_name"]].to_csv("output/hhs_names.csv", index=False)

In [20]:
hhs_merge.rename(
    columns={
        "total_staffed_adult_icu_beds": "icu_beds_now",
        "icu_beds": "icu_beds_then",
    },
    inplace=True,
)

---

In [21]:
hhs_merge["icu_beds_change"] = hhs_merge["icu_beds_now"] - hhs_merge["icu_beds_then"]

In [22]:
hhs_merge["icu_beds_pct_change"] = round(
    (hhs_merge["icu_beds_now"] - hhs_merge["icu_beds_then"])
    / hhs_merge["icu_beds_then"]
    * 100
)

### PCT change in ICU capacity before/after pandemic

In [23]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"][
    [
        "hospital",
        "county_x",
        "icu_beds_then",
        "icu_beds_now",
        "icu_beds_change",
        "icu_beds_pct_change",
    ]
].sort_values("icu_beds_change", ascending=False).head(20)

,hospital,county_x,icu_beds_then,icu_beds_now,icu_beds_change,icu_beds_pct_change
37,CENTINELA HOSPITAL MEDICAL CENTER,Los Angeles,19.0,85.0,66.0,347.0
216,POMONA VALLEY HOSPITAL MEDICAL CENTER,Los Angeles,38.0,84.0,46.0,121.0
267,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,Los Angeles,5.0,50.0,45.0,900.0
100,HUNTINGTON MEMORIAL HOSPITAL,Los Angeles,38.0,74.0,36.0,95.0
70,EMANATE HEALTH INTER-COMMUNITY HOSPITAL,Los Angeles,8.0,40.0,32.0,400.0
153,LAKEWOOD REGIONAL MEDICAL CENTER,Los Angeles,8.0,40.0,32.0,400.0
217,PROVIDENCE HOLY CROSS MEDICAL CENTER,Los Angeles,12.0,43.0,31.0,258.0
121,KAISER FOUNDATION HOSPITAL - SOUTH BAY,Los Angeles,20.0,49.0,29.0,145.0
125,KAISER FOUNDATION HOSPITAL - WEST LA,Los Angeles,31.0,58.0,27.0,87.0
115,KAISER FOUNDATION HOSPITAL - PANORAMA CITY,Los Angeles,24.0,47.0,23.0,96.0


### In late 2019, hospitals in LA County reported this many ICU beds

In [24]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds"].sum()

KeyError: 'icu_beds'

### According to latest HHS reports, they've added this many

In [ ]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds_change"].sum()

### Total ICU beds now

In [ ]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["total_staffed_adult_icu_beds"].sum()

### PCT increase in ICU beds? 

In [ ]:
round(
    (
        (
            hhs_merge[hhs_merge["county_y"] == "Los Angeles"][
                "total_staffed_adult_icu_beds"
            ].sum()
            - hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds"].sum()
        )
        / hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds"].sum()
        * 100
    ),
    2,
)